In [1]:
import pandas as pd
from os import listdir
import numpy as np

In [2]:
directory = "ados_datasets/"

In [18]:
filename='current_data.txt'
txtname = directory+filename
data = pd.read_csv(txtname, sep="\t")

In [19]:
potentiallyUseful = ['ados diagnosis classification',
        'age in months at the time of the interview/test/sampling/imaging.',
        'anxiety',
        'hand and finger and other complex mannerisms',
        'imagination/creativity', 
        'immediate echolalia',
        'quality of social overtures', 
        'self-injurious behavior',
        'sex of the subject', 
        'shared enjoyment in interaction',
        'tantrums, aggression, negative or disruptive behavior',
        'unusual eye contact',
        'diagnosis']

In [20]:
data[potentiallyUseful].shape

(4072, 13)

In [21]:
print ('Number of rows with at least 1 NaN features')
print (data[data.isnull().sum(axis=1) > 0].shape[0])
# Remove them for now
data = data[data.isnull().sum(axis=1) == 0]
print (str(data.shape[0]) + ' rows left')

Number of rows with at least 1 NaN features
1339
2733 rows left


In [22]:
for col in data[potentiallyUseful]:
    print("COLUMN:", col)
    print(data[col].unique())

COLUMN: ados diagnosis classification
['Non-Spectrum' 'non-spectrum' 'ASD' 'Non-spectrum' 'non spectrum'
 'Autism' 'autism' 'none' 'autism spectrum' 'Unknown' 'Autism Spectrum'
 'non' 'aut' '2' '0' '1' 'nonspectrum/typical' 'typical' 'nonspec'
 'Typical' 'non-spec' 'Non spectrum' 'Non-Spec' 'Autism spectrum'
 'Nonspectrum' 'Non Spectrum' 'nonspectrum' 'Typical Non spectrum'
 'spectrum' 'non asd' 'Autistic' 'Typical, non-spectrum' 'autismspec'
 'typical/non-spectrum' 'non spec' 'non-spect' 'typical-nonspectrum'
 'ASD spectrum' '9' 'autism spect' 'Autism Spectrum.' 'Nonspectrum.'
 'Autism Spectrum?' 'Autism.' 'Non-spectrum.' 'Autism Spectrum/P' '3']
COLUMN: age in months at the time of the interview/test/sampling/imaging.
[ 35  30  23  31  13  28  26  41  24  38  32  33  34  37  43  36  19  69
  20  25  40  21  46  45  39  29  22  53  27  50  44  42  47  51  49  18
  48  87   7 224  81  95  75  77 101  83  63 149  79  99 203  61  15 194
 125 176  88 148  56  70  60  62 144 146 178 147 11

In [8]:
data.corr()

,age in months at the time of the interview/test/sampling/imaging.,anxiety,collection_id,dataset_id,hand and finger and other complex mannerisms,imagination/creativity,immediate echolalia,quality of social overtures,self-injurious behavior,shared enjoyment in interaction,"tantrums, aggression, negative or disruptive behavior",unusual eye contact,diagnosis
age in months at the time of the interview/test/sampling/imaging.,1.000000,-0.090063,0.544941,0.130405,0.131410,0.034405,-0.026138,0.203442,0.116732,0.220893,-0.087972,0.236806,-0.139418
anxiety,-0.090063,1.000000,-0.065304,-0.008045,0.002741,0.076958,0.021019,0.026470,-0.008196,0.022011,0.276912,-0.016306,0.071808
collection_id,0.544941,-0.065304,1.000000,0.619944,0.145397,-0.007391,0.004802,0.080201,0.058374,0.110759,-0.071230,0.192137,-0.278930
dataset_id,0.130405,-0.008045,0.619944,1.000000,-0.083200,-0.162072,-0.088410,-0.156057,-0.041978,-0.098511,-0.067263,-0.030828,-0.267841
hand and finger and other complex mannerisms,0.131410,0.002741,0.145397,-0.083200,1.000000,0.408598,0.369820,0.547893,0.175715,0.401037,0.146584,0.475258,0.280266
imagination/creativity,0.034405,0.076958,-0.007391,-0.162072,0.408598,1.000000,0.452513,0.533556,0.138497,0.477102,0.183007,0.437986,0.255464
immediate echolalia,-0.026138,0.021019,0.004802,-0.088410,0.369820,0.452513,1.000000,0.426051,0.123451,0.353349,0.110808,0.314560,0.205751
quality of social overtures,0.203442,0.026470,0.080201,-0.156057,0.547893,0.533556,0.426051,1.000000,0.190682,0.641165,0.262117,0.681350,0.483629
self-injurious behavior,0.116732,-0.008196,0.058374,-0.041978,0.175715,0.138497,0.123451,0.190682,1.000000,0.155224,0.128919,0.176406,0.065548
shared enjoyment in interaction,0.220893,0.022011,0.110759,-0.098511,0.401037,0.477102,0.353349,0.641165,0.155224,1.000000,0.237177,0.510550,0.286652


In [11]:
data['diagnosis'].value_counts()

 0    1739
 2     803
 1     104
-1      87
Name: diagnosis, dtype: int64

### Ricardo - what you had written and I did not use

In [15]:
data=data.drop(['Unnamed: 0', 'collection_id', 'collection_title', 'dataset_id', 'promoted_subjectkey', 'the ndar global unique identifier (guid) for research subject'], axis=1)

In [30]:
data1=data[(data['ados diagnosis classification']=='ASD') | (data['ados diagnosis classification']=='non-spectrum')]

In [33]:
data1[['ados diagnosis classification', 'imagination/creativity']]

,ados diagnosis classification,imagination/creativity
1,non-spectrum,2.0
2,non-spectrum,1.0
3,non-spectrum,0.0
4,non-spectrum,1.0
5,ASD,2.0
6,ASD,2.0
8,ASD,2.0
9,ASD,3.0
10,ASD,1.0
11,non-spectrum,2.0
